In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import aiohttp
import asyncio  
import nest_asyncio
import csv  

In [5]:
nest_asyncio.apply()
async def fetch_url(session, url):
    async with session.get(url) as response:
        return await response.text()

async def get_product_async(link):
    url = link + "?sort=price.net_asc"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    async with aiohttp.ClientSession(headers=headers) as session:
        response_text = await fetch_url(session, url)
        return response_text
def get_product(link):
    loop = asyncio.get_event_loop()
    request_start_time = time.time()
    response_text = loop.run_until_complete(get_product_async(link)) 
    request_end_time = time.time()
    print(f"Thời gian gửi yêu cầu HTTP: {request_end_time - request_start_time:.2f} giây")
    parse_start_time = time.time()
    soup = BeautifulSoup(response_text, 'html.parser')
    parse_end_time = time.time()
    print(f"Thời gian phân tích HTML: {parse_end_time - parse_start_time:.2f} giây")
    result = []
    product_list = soup.select('.zA.oY.pu.rg.zB.cv-auto.kF.kG.kI, .zA.oY.pu.rg.zB.cv-auto.kF.kG.kH')
    loop_start_time = time.time()
    for id, product in enumerate(product_list):
        product_name = product.select_one('h3.zE.hd').get_text(strip=True) if product.select_one('h3.zE.hd') else None
        product_price = product.select_one('div.z2').get_text(strip=True) if product.select_one('div.z2') else None
        product_image = product.select_one('img.z9.z6.l-')['src'] if product.select_one('img.z9.z6.l-') else None
        result.append({
            'id': id,
            'product_name': product_name,
            'product_price': product_price,
            'product_image': product_image,
        })
    loop_end_time = time.time()
    print(f"Thời gian duyệt qua các sản phẩm: {loop_end_time - loop_start_time:.2f} giây")
    count_dict = len(result)
    file_save_start_time = time.time()
    with open('categories_v1.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
    file_save_end_time = time.time()
    print(f"Thời gian lưu dữ liệu vào file JSON: {file_save_end_time - file_save_start_time:.2f} giây")
    csv_file_save_start_time = time.time()
    with open('categories_v1.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['id', 'product_name', 'product_price', 'product_store', 'product_image', 'product_rating', 'api'])
        writer.writeheader()
        writer.writerows(result)
    csv_file_save_end_time = time.time()
    print(f"Thời gian lưu dữ liệu vào file CSV: {csv_file_save_end_time - csv_file_save_start_time:.2f} giây")
    print("Dữ liệu đã được lưu vào file `categories_v1.json` và có " + str(count_dict))



In [6]:
get_product("https://iprice.vn/dien-thoai-may-tinh-bang/dien-thoai-thong-minh/")

Thời gian gửi yêu cầu HTTP: 0.56 giây
Thời gian phân tích HTML: 0.05 giây
Thời gian duyệt qua các sản phẩm: 0.01 giây
Thời gian lưu dữ liệu vào file JSON: 0.00 giây
Thời gian lưu dữ liệu vào file CSV: 0.00 giây
Dữ liệu đã được lưu vào file `categories_v1.json` và có 59
